In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/mink/notebooks/CameraTraps')  # append this repo to PYTHONPATH
sys.path.append('/home/mink/lib/ai4eutils')

In [3]:
import json
import os
from collections import defaultdict
from random import sample
from shutil import copyfile
from datetime import datetime, date
from tqdm import tqdm
from unidecode import unidecode 

# ai4eutils
import path_utils

from data_management.megadb.schema import sequences_schema_check
from data_management.megadb.converters.cct_to_megadb import make_cct_embedded, process_sequences, write_json

# alka_squirrels

The ideal dataset has both **location** and **sequence** information, in addition to any species or bounding box labels.

Give the path to a JSON file where output from this script will be written to. You can then take this file to the .Net app for ingestion to the database.

In [4]:
path_to_output = '/home/mink/camtraps/data/megadb_jsons/alka_squirrels.json' 
path_to_output_temp = '/home/mink/camtraps/data/megadb_jsons/alka_squirrels_temp.json' 

In [5]:
dataset_name = 'alka_squirrels'

## Step 0 - Add an entry to the `datasets` table

Done

## Step 1 - Prepare the `sequence` objects to insert into the database

### Step 1b - If you're starting from scratch...

i.e. starting from metadata in another format, e.g. spreadsheets, CSVs, 

you would need to compile the metadata into a list of `sequence` items - format specified at https://github.com/ecologize/CameraTraps/tree/master/data_management/megadb.

Call the result `sequences`

In [6]:
# all content in the container was first AzCopied to mink_disk_0
container_root = '/mink_disk_0/camtraps/alka_squirrels/alka/'
os.listdir(container_root)

['2', 'tagged', 'ALKA2018_cameratrap_miroslav', '1', 'annotations']

In [7]:
folder1 = '/mink_disk_0/camtraps/alka_squirrels/alka/1'
folder2 = '/mink_disk_0/camtraps/alka_squirrels/alka/2'
folder3 = '/mink_disk_0/camtraps/alka_squirrels/alka/ALKA2018_cameratrap_miroslav'
folder4 = '/mink_disk_0/camtraps/alka_squirrels/alka/tagged'

In [25]:
extensions = set([p.split('.')[-1] 
                  for p in path_utils.recursive_file_list(container_root, convert_slashes=False) 
                  if path_utils.is_image_file(p)])
extensions

{'JPG', 'jpg'}

### Folders 1 and 2

In [8]:
paths12 = path_utils.recursive_file_list(folder1, convert_slashes=False)
len(paths12)
paths12.extend(path_utils.recursive_file_list(folder2, convert_slashes=False))
len(paths12)

7805

34873

In [10]:
seq_dict = defaultdict(list)

for p in tqdm(paths12):
    
    fn = os.path.basename(p).split('.')[0]
    frame = fn.split('C')[1]
    seq_id = p.split('/')[-2].replace(' ', '_')  # same as location
    
    parts = fn.split(' ')
    year = int(parts[0])
    month = int(parts[1])
    day = int(parts[2])
    
    seq_dict[seq_id].append({
        'file': p.split(container_root)[1],
        'class': ['unlabeled'],
        'frame_num': int(frame),
        'datetime': str(date(year=year, month=month, day=day))
    })

100%|██████████| 34873/34873 [00:00<00:00, 166282.19it/s]


In [11]:
seq_dict.keys()

dict_keys(['2020_05_07_54C', '2020_05_07_50C', '2020_05_08_32C', '2020_05_07_48C', '2020_05_08_45C', '2020_05_08_35C', '2020_05_08_38C', '2020_05_08_25C', '2020_05_08_39C', '2020_05_08_42C', '2020_05_08_30C', '2020_05_07_47C', '2020_05_07_35C', '2020_05_07_25C', '2020_05_07_30C', '2020_05_07_32C'])

In [12]:
seq_dict['2020_05_08_42C'][-1]

{'file': '2/2020 05 08 42C/2020 05 08 42C102621.JPG',
 'class': ['unlabeled'],
 'frame_num': 102621,
 'datetime': '2020-05-08'}

### Folder ALKA2018_cameratrap_miroslav

In [13]:
seq_id = 'ALKA2018_miroslav'
seq_dict[seq_id] = []

for fn in os.listdir(folder3):
    if path_utils.is_image_file(fn):
    
        p = os.path.join(folder3, fn)
        frame = fn.split('.')[0].split('__')[1]
        
        seq_dict[seq_id].append({
            'file': p.split(container_root)[1],
            'class': ['unlabeled'],
            'frame_num': int(frame),
            'datetime': str(date(year=int(fn[:4]), month=int(fn[4:6]), day=int(fn[6:8])))
        })

In [14]:
seq_dict['ALKA2018_miroslav'][-10]

{'file': 'ALKA2018_cameratrap_miroslav/20180608NBR10L__01447.JPG',
 'class': ['unlabeled'],
 'frame_num': 1447,
 'datetime': '2018-06-08'}

### Folder tagged

Don't re-run the cell below

In [15]:
seq_id_prefix = 'tagged'

for fn in os.listdir(folder4):
    if path_utils.is_image_file(fn):
    
        p = os.path.join(folder4, fn)
        name = fn.split('.')[0]
        
        seq_id = seq_id_prefix + '_' + name.split('_')[0]
        frame = name.split('_')[1]
        
        seq_dict[seq_id].append({
            'file': p.split(container_root)[1],
            'class': ['unlabeled'],
            'frame_num': int(frame),
            'datetime': str(date(year=2018, month=6, day=7))  # seems to be all from this day
        })

In [16]:
seq_dict['tagged_9'][-10]

{'file': 'tagged/9_2461.jpg',
 'class': ['unlabeled'],
 'frame_num': 2461,
 'datetime': '2018-06-07'}

### Combine folders

In [17]:
sequences = []
num_images = 0

for seq_id, seq_images in seq_dict.items():
    sequences.append({
        'dataset': dataset_name,
        'seq_id': seq_id,
        'location': seq_id,
        'images': seq_images
    })
    num_images += len(seq_images)

len(sequences)
num_images

27

37449

## Step 2 - Pass the schema check

Once your metadata are in the MegaDB format for `sequence` items, we check that they conform to the format's schema.

If the format conforms, the following messages will be printed:

```
Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
```

For large datasets, the second step will take some time (~ a minute). 

Otherwise there will be an error message describing what's wrong. Please fix the issues until all checks are passed. You might need to write some snippets of code to loop through the `sequence` items to understand which entries have problems.

In [18]:
%%time

sequences_schema_check.sequences_schema_check(sequences)

Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
CPU times: user 1.9 s, sys: 0 ns, total: 1.9 s
Wall time: 1.9 s


Save the entries while we wait for the annotations to come back.

In [19]:
with open(path_to_output_temp, 'w', encoding='utf-8') as f:
    json.dump(sequences, f, indent=1, ensure_ascii=False)

### Step 2b - copy images to flat folder

In [26]:
dst_paths = []

for seq in tqdm(sequences):
    seq_id = seq['seq_id']
    for im in seq['images']:
        src_path = os.path.join(container_root, im['file'])
        assert os.path.exists(src_path), src_path
        frame = im['frame_num']
        dst_path = os.path.join('/mink_disk_0/camtraps/imerit12', f'{dataset_name}.seq{seq_id}.frame{frame}.jpg')
        
        dst_paths.append(copyfile(src_path, dst_path))
        
len(dst_paths)

100%|██████████| 27/27 [1:24:56<00:00, 188.76s/it]  


37449